# Large Scale Time Series Modelling with Fugue

This tutorial will be about scaling time series modelling to Spark, Dask, and Ray. When dealing with large scale data, there are two approaches users can take. The first one is they can train a model for the whole dataset. The approach is training a model for each inidividual timeseries. 

Timeseries data is very friednly to distributed computing because we normally have hundreds or thousands of relatively independent data points. We can pre-process and model each one independently. This set-up allows choosing the best model for each timeseries. If we have 100 time series, it's very possible that 50 will be better modelled with ARIMA and another 50 will be better modelled with ETS.

Related article: [Distributed Forecast of 1M Time Series in under 15 mins with Spark, Nixtla, and Fugue](https://towardsdatascience.com/distributed-forecast-of-1m-time-series-in-under-15-minutes-with-spark-nixtla-and-fugue-e9892da6fd5c)

The main tools used in this tutorial are:

[Nixtla](https://github.com/Nixtla) - The Nixtla project is focused lightning fast state-of-the-art timeseries modelling. The project has a few libraries
* [statsforecast](https://github.com/Nixtla/statsforecast) - focused on statistic and econometric models such as ARIMA, ETS 

[Fugue](https://github.com/fugue-project/fugue/) - an abstraction layer for Spark, Dask, and Ray. Fugue ports code written for local execution to distributed execution.

[Spark](https://github.com/apache/spark) on [Databricks](https://www.databricks.com/) - a distributed computing engine built on top of the JVM.


## Goals 

1. Learn how to deal with large scale data effectively
2. Demostrate a distributed model training available for every logical group of data
3. Show some SOTA timeseries forecasting work

Not covered:
1. Time series models for specific use cases (sparse, 0's, irregular)
2. Time series basics

# Tooling

![architecture](../img/architecture.png)

## First Look at Nixtla

We're going to take a quick look at Nixtla to understand the form data needs to be. 

In [ ]:
import pandas as pd
from statsforecast.utils import generate_series

series = generate_series(n_series=2, seed=1)

In [ ]:
series.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axs = plt.subplots(ncols=2, figsize=(16,4))
series = series.reset_index()
for unique_id in range(2):
    plt.figure()
    _temp = series.loc[series['unique_id'] == unique_id]
    sns.lineplot(x=_temp['ds'], y=_temp['y'], ax=axs[unique_id])

We need to reset the index for distributed backends.

ETS is doing 15 models and AutoARIMA is doing 100-something models.

In [ ]:
from statsforecast.models import AutoARIMA, ETS
from statsforecast.core import StatsForecast

sf = StatsForecast(df=series,
                   models=[AutoARIMA(), ETS()], 
                   freq='D', 
                   n_jobs=-1)

forecasts = sf.forecast(7)
forecasts.head()

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(16,4))
combined = pd.concat(
    [forecasts.reset_index().melt(id_vars=["unique_id","ds"]).rename(columns={"variable":"model", "value": "y"}),
    series.assign(model="truth")]
    , axis=0, ignore_index=True)

for unique_id in range(2):
    plt.figure()
    _temp = combined.loc[combined['unique_id'] == unique_id]
    sns.lineplot(x=_temp['ds'], y=_temp['y'], hue=_temp['model'], ax=axs[unique_id])

## Fit-Predict Interface

With classical machine learning models, it's a very common approach to save the model weights with something like pickle to be loaded during prediction time. This is less common for timeseries modelling because the focus is on a lightweight train-predict when predictions are needed. This is what the `forecast()` method does.

Still, Nixtla has a `scikit-learn` type `fit-predict()` interface. Below is what it would look like.

In [ ]:
model = StatsForecast(df=series,
                      models=[AutoARIMA(), ETS()], 
                      freq='D', 
                      n_jobs=-1)
model.fit()

The predict function will just take in a horizon. 

There is an overhead to test multiple models.

In [ ]:
model.predict(h=7)

After the fitting, the `StatsForecast` object will contain the fitted models for each unique id. Note that you can't train on timeseries A, and predict on a different timeseries B (yet).

In [ ]:
model.fitted_

In [ ]:
model.fitted_[0][0].model_

## FugueBackend to Run on Spark, Dask, and Ray

In [ ]:
series = generate_series(n_series=50, seed=1).reset_index()
series['unique_id'] = series['unique_id'].astype(int)
series

In [ ]:
from pyspark.sql import SparkSession
from statsforecast.distributed.utils import forecast
from statsforecast.distributed.fugue import FugueBackend

spark = SparkSession.builder.getOrCreate()

backend = FugueBackend(spark)
result = forecast(series, 
                  models=[AutoARIMA()], 
                  freq="D", 
                  h=7, 
                  parallel=backend)

In [ ]:
type(result)

In [ ]:
result.show(5)

## Next Steps

In this section, we took an initial look how to use Nixtla's Statsforecat. In the next section, we'll begin applying it to a problem.